In [20]:
from openpyxl import load_workbook
import pandas as pd
from datetime import datetime

In [10]:
def read_worksheet(workbook, worksheet):
    '''
    Generic function that returns a worksheet object

    :param workbook: str containing the workbook name and path
    :param worksheet: str, worksheet name
    :return: worksheet object from openpyxl
    '''
    wb = load_workbook(workbook)
    return wb[worksheet]

In [11]:
def df_pivot(pivot_cache):
    '''
    Generic Function that returns both a DataFrame with the pivot table records and its dimensions in a dictionary

    :param pivot_cache: cacheDefinitions object from openpyxl
    :returns    DataFrame object: a pandas dataframe containing record's values and cacheFields names in columns
                dict_dim: dictionary containing all dimensions names and items from sharedItems fields
    '''
    record = []
    dims = {}
    dict_dim = {}
    for cf in pivot_cache.cacheFields:
        dims[cf.name] = cf.name
        dim_items = {}
        i = 0
        # let's get other items under this dimension
        for si in cf.sharedItems._fields:
            try:  # sometimes the list is empty
                dim_items[i] = si.v
                i += 1
            except:
                dim_items[i] = None
        dict_dim[cf.name] = dim_items

    # so now working with actual records
    for rows in pivot_cache.records.r:
        row = []
        for cols in rows._fields:
            try:  # we have to handle missing objects
                row.append(cols.v)
            except :
                row.append(None)
        record.append(row)
    # Let's turn it into a dataframe to work properly
    return pd.DataFrame(columns=dims, data=record), dict_dim

In [16]:
def remap(df, dict_dim):
    '''
    Generic function to remap fact table with dimensions from cacheDefinitions

    :param df: DataFrame object containing the fact table from record's values
    :param dict_dim: dictionary containing all dimensions names and items from sharedItems fields
    :return: DataFrame object containing actual (human readable) data
    '''
    remapped_columns = {}
    for c in df:
        def change_value(x):
            # Function that returns a given dictionary changing only values
            # that are not nan (avoids use of lambda)
            return dict_dim[c].get(x, x)
        c2 = df[c].apply(change_value)
        remapped_columns[c] = c2
    return df.assign(**remapped_columns)

In [18]:
def build_schema(nice_table):
    '''
    Specific Function that builds a given schema

    :param nice_table: a DataFrame object containing specific pivot table schema
    :return: DataFrame object with following schema:
    Column	    Type
    year_month	date
    uf	        string
    product	    string
    unit	    string
    volume	    double
    created_at	timestamp
    '''
    nice_table_schema = pd.melt(nice_table,
                                id_vars=['COMBUSTÍVEL', 'REGIÃO', 'ANO', 'ESTADO', 'UNIDADE', 'TOTAL'],
                                var_name='month',
                                value_name='volume')
    nice_table_schema.drop(['REGIÃO', 'TOTAL'], axis=1, inplace=True)
    nice_table_schema = nice_table_schema.rename(columns={"COMBUSTÍVEL": "product",
                                                          "ANO": "year_month",
                                                          "ESTADO": "uf",
                                                          "UNIDADE": "unit"})

    # Changing month literals for numbers in MM format
    months = {
        'Jan': '01',
        'Fev': '02',
        'Mar': '03',
        'Abr': '04',
        'Mai': '05',
        'Jun': '06',
        'Jul': '07',
        'Ago': '08',
        'Set': '09',
        'Out': '10',
        'Nov': '11',
        'Dez': '12',
    }
    nice_table_schema['month'].replace(months, inplace=True)
    data_types_dict = {'year_month': int}  # take this floating point off of my sight!
    nice_table_schema = nice_table_schema.astype(data_types_dict)
    data_types_dict = {'year_month': str}  # now we can concatenate in a easier way
    nice_table_schema = nice_table_schema.astype(data_types_dict)
    nice_table_schema['year_month'] = nice_table_schema['year_month'] + nice_table_schema['month']
    nice_table_schema['year_month'] = pd.to_datetime(nice_table_schema['year_month'], format="%Y%m")  # date format!
    nice_table_schema['created_at'] = pd.Timestamp(datetime.now())
    nice_table_schema.drop('month', axis=1, inplace=True)  # drop mic... thug life!
    return nice_table_schema

In [12]:
ws = read_worksheet(workbook="vendas-combustiveis-m3.xlsx", worksheet="Plan1")
pvt_cache = ws._pivots[3].cache
df1_aux = df_pivot(pvt_cache)

In [14]:
df1_aux[0]

,COMBUSTÍVEL,ANO,REGIÃO,ESTADO,UNIDADE,Jan,Fev,Mar,Abr,Mai,Jun,Jul,Ago,Set,Out,Nov,Dez,TOTAL
0,0,0,0,0,m3,9563.263000,11341.229000,9369.746000,10719.983000,11165.968000,12312.451000,11220.970000,12482.281000,13591.122000,11940.570,11547.576,10818.094,136073.253000
1,0,0,0,1,m3,3065.758000,3495.290000,2946.930000,3023.920000,3206.930000,3612.580000,3264.460000,3835.740000,3676.571000,3225.610,3289.718,3358.346,40001.853000
2,0,0,0,2,m3,17615.604000,20258.200000,18741.344000,19604.023000,20221.674000,20792.616000,19912.898000,21869.338000,21145.643000,20633.175,20766.918,21180.919,242742.352000
3,0,0,0,3,m3,3259.300000,3636.216000,3631.569000,3348.416000,3394.016000,4078.616000,3346.616000,4029.900000,4358.516000,3716.032,3200.400,3339.332,43338.929000
4,0,0,0,4,m3,28830.479000,32297.047000,27310.979000,29396.384000,26511.009000,36553.250000,31807.840000,31009.972000,29755.907000,28661.951,28145.784,29294.796,359575.398000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4531,7,20,3,22,m3,58679.211957,57649.414855,68522.800725,64045.161232,66749.367754,73247.920290,81469.222826,75607.009058,72818.829710,NaN,NaN,NaN,618788.938406
4532,7,20,4,23,m3,13774.106884,14137.695652,16281.139493,15037.528986,15190.643116,15877.811594,16876.476449,16036.112319,14687.788043,NaN,NaN,NaN,137899.302536
4533,7,20,4,24,m3,18763.161232,18321.987319,20677.483696,19118.619565,18773.081522,19899.867754,20678.385870,18972.943841,18051.436594,NaN,NaN,NaN,173256.967391
4534,7,20,4,25,m3,46850.585145,46581.878623,53302.086957,48894.621377,47184.788043,49687.793478,52665.449275,50069.005435,48032.530797,NaN,NaN,NaN,443268.739130


In [15]:
df1_aux[1]

{'COMBUSTÍVEL': {0: 'GASOLINA C (m3)',
  1: 'GASOLINA DE AVIAÇÃO (m3)',
  2: 'QUEROSENE ILUMINANTE (m3)',
  3: 'QUEROSENE DE AVIAÇÃO (m3)',
  4: 'ÓLEO DIESEL (m3)',
  5: 'ÓLEO COMBUSTÍVEL (m3)',
  6: 'ETANOL HIDRATADO (m3)',
  7: 'GLP (m3)'},
 'ANO': {0: 2000.0,
  1: 2001.0,
  2: 2002.0,
  3: 2003.0,
  4: 2004.0,
  5: 2005.0,
  6: 2006.0,
  7: 2007.0,
  8: 2008.0,
  9: 2009.0,
  10: 2010.0,
  11: 2011.0,
  12: 2012.0,
  13: 2013.0,
  14: 2014.0,
  15: 2015.0,
  16: 2016.0,
  17: 2017.0,
  18: 2018.0,
  19: 2019.0,
  20: 2020.0},
 'REGIÃO': {0: 'REGIÃO NORTE',
  1: 'REGIÃO NORDESTE',
  2: 'REGIÃO SUDESTE',
  3: 'REGIÃO SUL',
  4: 'REGIÃO CENTRO-OESTE'},
 'ESTADO': {0: 'RONDÔNIA',
  1: 'ACRE',
  2: 'AMAZONAS',
  3: 'RORAIMA',
  4: 'PARÁ',
  5: 'AMAPÁ',
  6: 'TOCANTINS',
  7: 'MARANHÃO',
  8: 'PIAUÍ',
  9: 'CEARÁ',
  10: 'RIO GRANDE DO NORTE',
  11: 'PARAÍBA',
  12: 'PERNAMBUCO',
  13: 'ALAGOAS',
  14: 'SERGIPE',
  15: 'BAHIA',
  16: 'MINAS GERAIS',
  17: 'ESPÍRITO SANTO',
  18: 'RIO DE J

In [17]:
Table1 = remap(df1_aux[0], df1_aux[1])
Table1

,COMBUSTÍVEL,ANO,REGIÃO,ESTADO,UNIDADE,Jan,Fev,Mar,Abr,Mai,Jun,Jul,Ago,Set,Out,Nov,Dez,TOTAL
0,GASOLINA C (m3),2000.0,REGIÃO NORTE,RONDÔNIA,m3,9563.263000,11341.229000,9369.746000,10719.983000,11165.968000,12312.451000,11220.970000,12482.281000,13591.122000,11940.570,11547.576,10818.094,136073.253000
1,GASOLINA C (m3),2000.0,REGIÃO NORTE,ACRE,m3,3065.758000,3495.290000,2946.930000,3023.920000,3206.930000,3612.580000,3264.460000,3835.740000,3676.571000,3225.610,3289.718,3358.346,40001.853000
2,GASOLINA C (m3),2000.0,REGIÃO NORTE,AMAZONAS,m3,17615.604000,20258.200000,18741.344000,19604.023000,20221.674000,20792.616000,19912.898000,21869.338000,21145.643000,20633.175,20766.918,21180.919,242742.352000
3,GASOLINA C (m3),2000.0,REGIÃO NORTE,RORAIMA,m3,3259.300000,3636.216000,3631.569000,3348.416000,3394.016000,4078.616000,3346.616000,4029.900000,4358.516000,3716.032,3200.400,3339.332,43338.929000
4,GASOLINA C (m3),2000.0,REGIÃO NORTE,PARÁ,m3,28830.479000,32297.047000,27310.979000,29396.384000,26511.009000,36553.250000,31807.840000,31009.972000,29755.907000,28661.951,28145.784,29294.796,359575.398000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4531,GLP (m3),2020.0,REGIÃO SUL,RIO GRANDE DO SUL,m3,58679.211957,57649.414855,68522.800725,64045.161232,66749.367754,73247.920290,81469.222826,75607.009058,72818.829710,NaN,NaN,NaN,618788.938406
4532,GLP (m3),2020.0,REGIÃO CENTRO-OESTE,MATO GROSSO DO SUL,m3,13774.106884,14137.695652,16281.139493,15037.528986,15190.643116,15877.811594,16876.476449,16036.112319,14687.788043,NaN,NaN,NaN,137899.302536
4533,GLP (m3),2020.0,REGIÃO CENTRO-OESTE,MATO GROSSO,m3,18763.161232,18321.987319,20677.483696,19118.619565,18773.081522,19899.867754,20678.385870,18972.943841,18051.436594,NaN,NaN,NaN,173256.967391
4534,GLP (m3),2020.0,REGIÃO CENTRO-OESTE,GOIÁS,m3,46850.585145,46581.878623,53302.086957,48894.621377,47184.788043,49687.793478,52665.449275,50069.005435,48032.530797,NaN,NaN,NaN,443268.739130


In [21]:
build_schema(Table1)

,product,year_month,uf,unit,volume,created_at
0,GASOLINA C (m3),2000-01-01,RONDÔNIA,m3,9563.263,2023-05-15 18:44:50.220022
1,GASOLINA C (m3),2000-01-01,ACRE,m3,3065.758,2023-05-15 18:44:50.220022
2,GASOLINA C (m3),2000-01-01,AMAZONAS,m3,17615.604,2023-05-15 18:44:50.220022
3,GASOLINA C (m3),2000-01-01,RORAIMA,m3,3259.300,2023-05-15 18:44:50.220022
4,GASOLINA C (m3),2000-01-01,PARÁ,m3,28830.479,2023-05-15 18:44:50.220022
...,...,...,...,...,...,...
54427,GLP (m3),2020-12-01,RIO GRANDE DO SUL,m3,NaN,2023-05-15 18:44:50.220022
54428,GLP (m3),2020-12-01,MATO GROSSO DO SUL,m3,NaN,2023-05-15 18:44:50.220022
54429,GLP (m3),2020-12-01,MATO GROSSO,m3,NaN,2023-05-15 18:44:50.220022
54430,GLP (m3),2020-12-01,GOIÁS,m3,NaN,2023-05-15 18:44:50.220022
